In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

The Protein Data Bank (PDB) stores data for various proteins, each protein being designated by a four character alphanumeric ID. Some of this data includes 3-D structure information; this data is important because the structure of a protein determines its function.

Even still, predicting the shape of a protein still remains challenging, though huge breakthroughs have been made in this field using [machine learning](https://pubmed.ncbi.nlm.nih.gov/34015749/). This Notebook serves as a brief exploration into the PDB and protein modeling.

In [ ]:
import urllib
import warnings
import Bio
from Bio import BiopythonWarning
from Bio.PDB import PDBParser
import ipywidgets as widgets
import nglview as nv

# Ignore Bio.PDBConstructionWarning for discontinuous protein chains
warnings.simplefilter('ignore', BiopythonWarning)

def visualize_protein(change):
    
    # Handle error if user re-selects default value
    if change['new'] == 'Select a protein...':
        with widget_output:
            print('Please select a protein to display.')
            widget_output.clear_output(wait=True)
        
    else:
        pdb_id = change['new']
        
        # Fetch and cache PDB data
        urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb_id}.pdb',
                                   f'pdb_data/{pdb_id}.pdb')
        # Create parser instance    
        parser = PDBParser()

        # Read in protein structure from stored PDB file
        structure = parser.get_structure(f'{pdb_id}',f'pdb_data/{pdb_id}.pdb')
    
        # Visualize struture with accompanying GUI
        view = nv.show_biopython(structure, gui=True)
    
        # Display widget output; clear display when output changes
        with widget_output:
            display(view)
            widget_output.clear_output(wait=True)

# Capture the output of widgets
widget_output = widgets.Output()

# Selector widget
protein_selector = widgets.Dropdown(
    options=['Select a protein...', '7CYQ', '4M6U', '1UBQ', '4ADS', '1ZQA'],
    value='Select a protein...',
    description='PDB ID:',
    disabled=False)

# Handle PDB ID selection from widget
protein_selector.observe(visualize_protein, names='value')

# Display selector widget, output
widgets.VBox([protein_selector, widget_output])